In [1]:
!pip install transformers datasets torch torchaudio soundfile librosa
!pip3 install soundfile datasets pyctcdecode==v0.1.0
!pip3 install https://github.com/kpu/kenlm/archive/master.zip



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing

In [2]:
from pyctcdecode import Alphabet, BeamSearchDecoderCTC, LanguageModel


In [3]:
import kenlm

def get_decoder_ngram_model(tokenizer, ngram_lm_path):
    vocab_dict = tokenizer.get_vocab()
    sort_vocab = sorted((value, key) for (key, value) in vocab_dict.items())
    vocab = [x[1] for x in sort_vocab][:-2]
    vocab_list = vocab
    # convert ctc blank character representation
    vocab_list[tokenizer.pad_token_id] = ""
    # replace special characters
    vocab_list[tokenizer.unk_token_id] = ""
    # vocab_list[tokenizer.bos_token_id] = ""
    # vocab_list[tokenizer.eos_token_id] = ""
    # convert space character representation
    vocab_list[tokenizer.word_delimiter_token_id] = " "
    # specify ctc blank char index, since conventially it is the last entry of the logit matrix
    alphabet = Alphabet.build_alphabet(vocab_list, ctc_token_idx=tokenizer.pad_token_id)
    lm_model = kenlm.Model(ngram_lm_path)
    decoder = BeamSearchDecoderCTC(alphabet,
                                   language_model=LanguageModel(lm_model))
    return decoder

In [4]:
import transformers
print(transformers.__version__)


4.51.3


In [5]:
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor


2025-05-23 10:47:08.261721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747997228.461321      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747997228.518885      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
!ls /kaggle/input/checkpoint/results/wav2vec2-finetuned


added_tokens.json  config.json		     special_tokens_map.json
checkpoint-4000    model.safetensors	     tokenizer_config.json
checkpoint-6560    preprocessor_config.json  vocab.json


In [7]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model_path = "/kaggle/input/checkpoint/results/wav2vec2-finetuned"

processor = Wav2Vec2Processor.from_pretrained(model_path)

model = Wav2Vec2ForCTC.from_pretrained(
    model_path,
    low_cpu_mem_usage=True  # tuỳ chọn, giúp tiết kiệm RAM
)


In [9]:

lm_file = '/kaggle/input/checkpoint/vi_lm_4grams.bin'


ngram_lm_model = get_decoder_ngram_model(processor.tokenizer, lm_file)

In [10]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import soundfile as sf
import torch

# load model and tokenizer
# processor = Wav2Vec2Processor.from_pretrained("/kaggle/working/wav2vec2-finetuned")
# model = Wav2Vec2ForCTC.from_pretrained("/kaggle/working/wav2vec2-finetuned")

# define function to read in sound file
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

# load dummy dataset and read soundfiles
ds = map_to_array({
    "file": '/kaggle/input/vivos-vietnamese/vivos/test/waves/VIVOSDEV02/VIVOSDEV02_R106.wav'
})

# tokenize
input_values = processor(ds["speech"], return_tensors="pt", padding="longest").input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits[0]
logits1 = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits1, dim=-1)
transcription = processor.batch_decode(predicted_ids)
beam_search_output = ngram_lm_model.decode(logits.cpu().detach().numpy(), beam_width=500)


It is strongly recommended to pass the `sampling_rate` argument to `Wav2Vec2FeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


In [11]:
print(beam_search_output)

trở nên thụ động


In [12]:
print(transcription)

['trở nên thụ động']


In [13]:
import os

wav_dir = "/kaggle/input/asr-data/private_test3/cleaned_audio_0"
audio_files = []

for root, dirs, files in os.walk(wav_dir):
    for file in files:
        if file.endswith((".wav", ".mp3")):  # kiểm tra cả wav và mp3
            audio_files.append(os.path.join(root, file))

print(f"Tổng số file .wav và .mp3: {len(audio_files)}")
print("Ví dụ vài file đầu:")
for f in audio_files[:10]:
    print(f)



Tổng số file .wav và .mp3: 4268
Ví dụ vài file đầu:
/kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_0194.wav
/kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_3485.wav
/kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_0771.wav
/kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_0273.wav
/kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_2736.wav
/kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_2068.wav
/kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_0464.wav
/kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_0668.wav
/kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_1977.wav
/kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_0390.wav


In [ ]:
import os
import torch
import soundfile as sf
from pydub import AudioSegment
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Load model và processor
model.eval()

# Đường dẫn đến thư mục chứa audio
audio_dir = "/kaggle/input/asr-data/private_test/private-test-data-asr"
audio_files = []

# Lấy danh sách tất cả .wav và .mp3
for root, dirs, files in os.walk(audio_dir):
    for file in files:
        if file.endswith((".wav", ".mp3")):
            audio_files.append(os.path.join(root, file))

# Mở file để ghi kết quả
with open("transcrip6.txt", "w", encoding="utf-8") as out_file:
    for audio_path in audio_files:
        # Đọc file âm thanh (xử lý khác nhau tùy định dạng)
        if audio_path.endswith(".mp3"):
            audio = AudioSegment.from_mp3(audio_path)
            audio = audio.set_frame_rate(16000).set_channels(1)
            samples = audio.get_array_of_samples()
            speech = torch.tensor(samples, dtype=torch.float32).numpy() / (1 << 15)  # Chuẩn hóa [-1, 1]
        else:  # wav
            speech, _ = sf.read(audio_path)

        # Tokenize
        inputs = processor(speech, return_tensors="pt", padding=True)

        # Dự đoán logits
        with torch.no_grad():
            logits = model(inputs.input_values).logits

        # Lấy ID dự đoán
        predicted_ids = torch.argmax(logits, dim=-1)

        # Decode thành chữ
        transcription = processor.batch_decode(predicted_ids)[0]

        # Ghi vào file
        audio_id = os.path.splitext(os.path.basename(audio_path))[0]
        out_file.write(f"{audio_id}\t{transcription.strip()}\n")


In [ ]:
import os
import torchaudio
import torch
import soundfile as sf  # vẫn giữ nếu bạn chỉ dùng cho .wav
from tqdm import tqdm

model.eval()

wav_dir = "/kaggle/input/asr-data/private_test3/cleaned_audio_0"
audio_files = [
    os.path.join(root, f)
    for root, _, files in os.walk(wav_dir)
    for f in files if f.endswith((".wav", ".mp3"))
]

with open("transcrip.txt", "w", encoding="utf-8") as out_file:
    for audio_path in tqdm(audio_files):
        # 1) đọc âm thanh
        if audio_path.endswith(".wav"):
            speech, _ = sf.read(audio_path)
        else:  # nếu là mp3
            speech, sr = torchaudio.load(audio_path)  # Tensor [1, T]
            speech = speech.squeeze(0).numpy()  # chuyển thành numpy 1-D

        # 2) tokenize
        inputs = processor(speech, return_tensors="pt", padding=True)

        # 3) suy diễn logits cho file hiện tại
        with torch.no_grad():
            logits = model(inputs.input_values).logits[0]

        # 4) beam search decode với LM
        transcription = ngram_lm_model.decode(
            logits.cpu().numpy(), beam_width=1000
        )

        # 5) ghi ra file
        audio_id = os.path.splitext(os.path.basename(audio_path))[0]
        out_file.write(f"{audio_id}\t{transcription.strip()}\n")

print(f"✅ Đã ghi {len(audio_files)} dòng vào transcrip3.txt")


In [ ]:
!pip install librosa

In [ ]:
import os
import torchaudio
import torch
import soundfile as sf
import librosa
import numpy as np
from tqdm import tqdm

# Giả định: model, processor, và ngram_lm_model đã được khởi tạo
model.eval()

# Đường dẫn thư mục chứa file âm thanh
wav_dir = "/kaggle/input/asr-data/private_test3/cleaned_audio_0"
audio_files = [
    os.path.join(root, f)
    for root, _, files in os.walk(wav_dir)
    for f in files if f.endswith((".wav", ".mp3"))
]

# Tốc độ chuẩn hóa (ví dụ: làm chậm 20% -> rate = 0.8)
target_rate = 0.8  # Điều chỉnh dựa trên nhu cầu (0.7-0.9: chậm hơn, >1: nhanh hơn)

with open("transcrip.txt", "w", encoding="utf-8") as out_file:
    for audio_path in tqdm(audio_files):
        # 1) Đọc âm thanh
        if audio_path.endswith(".wav"):
            speech, sr = sf.read(audio_path)
        else:  # nếu là mp3
            speech, sr = torchaudio.load(audio_path)  # Tensor [1, T]
            speech = speech.squeeze(0).numpy()  # chuyển thành numpy 1-D

        # 2) Chuẩn hóa tốc độ nói
        try:
            normalized_speech = librosa.effects.time_stretch(speech, rate=target_rate)
        except Exception as e:
            print(f"Lỗi khi chuẩn hóa tốc độ cho {audio_path}: {e}")
            normalized_speech = speech  # Nếu lỗi, giữ nguyên âm thanh gốc

        # 3) Tokenize
        inputs = processor(normalized_speech, sampling_rate=sr, return_tensors="pt", padding=True)

        # 4) Suy diễn logits cho file hiện tại
        with torch.no_grad():
            logits = model(inputs.input_values).logits[0]

        # 5) Beam search decode với LM
        transcription = ngram_lm_model.decode(
            logits.cpu().numpy(), beam_width=1000
        )

        # 6) Ghi ra file
        audio_id = os.path.splitext(os.path.basename(audio_path))[0]
        out_file.write(f"{audio_id}\t{transcription.strip()}\n")

print(f"✅ Đã ghi {len(audio_files)} dòng vào transcrip.txt")

In [14]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.1 MB/s eta 0:00:0000:010:01
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4304 sha256=2e4a7520f4ba803adca41c5ac4248978b21bdeb364ac0c4c629e535b28f65574
  Stored in directory: /root/.cache/pip/wheels/6d/2d/d6/158260bfd6820d144535857b80cc112bee5c3aa6d81b6dc049
Successfully built py_vncorenlp


In [15]:
import os
import torch
import torchaudio
import soundfile as sf
import numpy as np
from tqdm import tqdm
import py_vncorenlp
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Cài đặt các thư viện cần thiết
!pip install transformers torchaudio soundfile py_vncorenlp tqdm
!apt-get install -y ffmpeg  # Cần cho xử lý MP3

# Khởi tạo mô hình wav2vec
try:
    # processor = Wav2Vec2Processor.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")
    # model = Wav2Vec2ForCTC.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")
    model.eval()
except Exception as e:
    print(f"Lỗi khi khởi tạo mô hình wav2vec: {e}")
    raise SystemExit("Dừng chương trình do lỗi mô hình.")

# Khởi tạo VnCoreNLP để tách từ
os.makedirs('/kaggle/working/vncorenlp', exist_ok=True)
py_vncorenlp.download_model(save_dir='/kaggle/working/vncorenlp')
try:
    vncorenlp = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/working/vncorenlp')
except ValueError:
    print("JVM đã chạy, sử dụng VnCoreNLP hiện tại.")

# Giả định: ngram_lm_model đã được khởi tạo (nếu không, dùng processor.decode)
# Nếu bạn dùng pyctcdecode cho ngram_lm_model, đảm bảo đã cài đặt:
# !pip install pyctcdecode
# from pyctcdecode import build_ctcdecoder
# ngram_lm_model = build_ctcdecoder(labels=processor.tokenizer.get_vocab(), kenlm_model_path="/kaggle/input/path_to_lm.arpa")

# Đường dẫn thư mục chứa file âm thanh
wav_dir = "/kaggle/input/asr-data/private_test3/cleaned_audio_0"
audio_files = [
    os.path.join(root, f)
    for root, _, files in os.walk(wav_dir)
    for f in files if f.endswith((".wav", ".mp3"))
]

if not audio_files:
    print(f"Không tìm thấy file âm thanh trong {wav_dir}. Kiểm tra lại đường dẫn.")
    raise SystemExit("Dừng chương trình do không có file âm thanh.")

# Xử lý và ghi kết quả
output_file = "/kaggle/working/transcrip1.txt"
with open(output_file, "w", encoding="utf-8") as out_file:
    for audio_path in tqdm(audio_files, desc="Processing audio files"):
        # 1) Đọc âm thanh
        try:
            if audio_path.endswith(".wav"):
                speech, sr = sf.read(audio_path)
            else:  # nếu là mp3
                speech, sr = torchaudio.load(audio_path)  # Tensor [1, T]
                speech = speech.squeeze(0).numpy()  # chuyển thành numpy 1-D
            print(f"Đã đọc file: {audio_path}")
        except Exception as e:
            print(f"Lỗi khi đọc file {audio_path}: {e}")
            continue

        # 2) Tokenize
        try:
            inputs = processor(speech, sampling_rate=sr, return_tensors="pt", padding=True)
            print(f"Đã tokenize file: {audio_path}")
        except Exception as e:
            print(f"Lỗi khi tokenize {audio_path}: {e}")
            continue

        # 3) Suy diễn logits
        try:
            with torch.no_grad():
                logits = model(inputs.input_values).logits[0]
            print(f"Đã suy diễn logits cho: {audio_path}")
        except Exception as e:
            print(f"Lỗi khi suy diễn logits cho {audio_path}: {e}")
            continue

        # 4) Beam search decode với LM (hoặc dùng processor nếu không có ngram_lm_model)
        try:
            transcription = ngram_lm_model.decode(logits.cpu().numpy(), beam_width=1000)
        except NameError:  # Nếu ngram_lm_model chưa được định nghĩa
            transcription = processor.batch_decode(logits.unsqueeze(0), num_beams=5)[0]
        except Exception as e:
            print(f"Lỗi khi decode {audio_path}: {e}")
            transcription = ""
        print(f"Transcript gốc: {transcription}")

        # 5) Tách từ bằng VnCoreNLP
        try:
            corrected_transcription = vncorenlp.word_segment(transcription)[0] if transcription else transcription
            print(f"Transcript sau tách từ: {corrected_transcription}")
        except Exception as e:
            print(f"Lỗi khi tách từ cho {audio_path}: {e}")
            corrected_transcription = transcription  # Nếu lỗi, giữ nguyên

        # 6) Thay dấu _ bằng khoảng cách
        corrected_transcription = corrected_transcription.replace("_", " ")
        print(f"Transcript sau thay dấu _: {corrected_transcription}")

        # 7) Ghi ra file
        audio_id = os.path.splitext(os.path.basename(audio_path))[0]
        out_file.write(f"{audio_id}\t{corrected_transcription.strip()}\n")
        out_file.flush()  # Đảm bảo ghi ngay lập tức vào file
        print(f"Đã ghi transcript cho: {audio_id}")

# Kiểm tra file đầu ra
if os.path.exists(output_file):
    print(f"File {output_file} đã được tạo. Nội dung:")
    !cat {output_file}
else:
    print(f"File {output_file} không được tạo. Kiểm tra lỗi ở các bước trên.")

print(f"✅ Xử lý hoàn tất. Đã ghi {len(audio_files)} dòng vào {output_file}")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 87 not upgraded.


--2025-05-23 10:48:32--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412703 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.2.jar’

     0K .......... .......... .......... .......... ..........  0%  891K 30s
    50K .......... .......... .......... .......... ..........  0% 3.00M 19s
   100K .......... .......... .......... .......... ..........  0% 1.42M 19s
   150K .......... .......... .......... .......... ..........  0% 3.63M 16s
   200K .......... .......... .......... .......... ..........  0% 6.20M 14s
   250K .......... .......... .......... .......... ..........  1% 2.03M 13s
   300K .......... .......... .......... .......... ..........  1% 8.39M 12s
   

2025-05-23 10:48:50 INFO  WordSegmenter:24 - Loading Word Segmentation model


Processing audio files:   0%|          | 0/4268 [00:00<?, ?it/s]

Đã đọc file: /kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_0194.wav
Đã tokenize file: /kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_0194.wav
Đã suy diễn logits cho: /kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_0194.wav
Transcript gốc: khoảng thứ hai từ năm đến mười lăm triệu vậy thì khoảng khách của nhóm thứ hai của bạn là mười triệu bức thứ ba trên mười lăm triệu vậy là trên mười lăm triệu đồng một tháng thì từ mười lăm đến một số rất lớn hóng tần bao nhiêu chính vì vậy cho nên


...

Processing audio files: 100%|█████████▉| 4267/4268 [2:10:03<00:02,  2.95s/it]

Đã suy diễn logits cho: /kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_2129.wav
Transcript gốc: rồi không đợi cho cô đáp
Transcript sau tách từ: rồi không đợi cho cô đáp
Transcript sau thay dấu _: rồi không đợi cho cô đáp
Đã ghi transcript cho: audio_2129
Đã đọc file: /kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_3855.wav
Đã tokenize file: /kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_3855.wav
Đã suy diễn logits cho: /kaggle/input/asr-data/private_test3/cleaned_audio_0/audio_3855.wav


Processing audio files: 100%|██████████| 4268/4268 [2:10:04<00:00,  1.83s/it]

Transcript gốc: anh nộp lại anh cần phải chăm sóc
Transcript sau tách từ: anh nộp lại anh cần phải chăm_sóc
Transcript sau thay dấu _: anh nộp lại anh cần phải chăm sóc
Đã ghi transcript cho: audio_3855
File /kaggle/working/transcrip1.txt đã được tạo. Nội dung:
audio_0194	khoảng thứ hai từ năm đến mười lăm triệu vậy thì khoảng khách của nhóm thứ hai của bạn là mười triệu bức thứ ba trên mười lăm triệu vậy là trên mười lăm triệu đồng một tháng thì từ mười lăm đến một số rất lớn hóng tần bao nhiêu chính vì vậy cho nên
audio_3485	một chút đây hôi này
audio_0771	rồi thế này sẽ lén tỉnh này thế thì đây là một phần ba lan ông nằm phía trên chục hành thôi đấy cuộc chi nhánh của ba ra bôn này thôi nhưng mà nằm trên trên chục hoạch thôi nó sẽ có hình thức này
audio_0273	sang phim khác thì vẫn là một cái con người đấy thì chắc chắn là không hay rồi
audio_2736	bên cạnh đó
audio_2068	hay là nói được câu nào cả
audio_0464	nhưng do có nhiều chuyện nên bán nhà đó để mua nơi khác
audio_0668	tháng mười

✅ Xử lý hoàn tất. Đã ghi 4268 dòng vào /kaggle/working/transcrip1.txt
